   PMD  SPOTBugs

In [10]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
import json
import pandas as pd

# Load the JSONL file
input_file_name = 'try.json'
output_file_name = 'output.csv'

# Load data from the JSONL file
data = []
with open(input_file_name, 'r') as file:
    for line in file:
        try:
            data.append(json.loads(line.strip()))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line: {line}")
            continue

# Initialize the CodeT5 model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

# Function to generate code for a single prompt
def generate_code(prompt):
    prompt = f"""
    Generate Java code for the following task. Return ONLY the code, without any explanations, comments, or additional text.
    Description: {prompt}
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    generated_ids = model.generate(input_ids, max_length=512)  # Adjust max_length as needed
    generated_code = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_code

# Prepare data for CSV
output_data = []

# Generate code for only the first 2 entries in the JSON data
for index, entry in enumerate(data[:2]):
    prompt = entry['nl']
    generated_code = generate_code(prompt)
    
    # Append the example number and generated Java code to the output data
    output_data.append({
        'Example No': index + 1,
        'Java Code': generated_code
    })

# Create a DataFrame from the output data
output_df = pd.DataFrame(output_data)

# Save the DataFrame to a CSV file
output_df.to_csv(output_file_name, index=False)

print(f"Generated Java codes saved to {output_file_name}")

Generated Java codes saved to output.csv
